In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline

tokenizer_GraphCodeBert = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")

model_GraphCodeBert = AutoModelForMaskedLM.from_pretrained("microsoft/graphcodebert-base")

/home/wolfram/miniconda3/envs/codecomp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import ast

def getLineAssignment(tree, lineno):
    return next((node for node in ast.walk(tree) if isinstance(node, ast.Name) and node.lineno == lineno), None)

def get_variables(expression):
    tree = ast.parse(expression)
    variables = []
    for node in ast.walk(tree):
        if not isinstance(node, ast.Assign):
            continue
        nodeVariable = getLineAssignment(tree, node.lineno).id
        if nodeVariable not in variables:
            variables.append(nodeVariable)
    return variables

In [3]:
class replaceName(ast.NodeTransformer):
    def __init__(self, varName, replaceName):
        self.varName = varName
        self.replaceName = replaceName

    def visit_Name(self, node):
        if(node.id == self.varName):
            node.id = self.replaceName
        return node


In [4]:
CODE = """
def Function(Variable):
    a = sin(Variable)
    b = cos(Variable)
    return a + b
"""

In [6]:
variables = get_variables(CODE)

tree = ast.parse(CODE)

for i in range(0, len(variables)):
    masked = ast.unparse(replaceName(variables[i], '<mask>').visit(ast.parse(CODE)))
    fill_mask_GCB = pipeline("fill-mask", model=model_GraphCodeBert, tokenizer=tokenizer_GraphCodeBert)
    candidates = fill_mask_GCB(masked)
    for j in range(0, len(candidates)):
        if candidates[0][j]['token_str'][1:] in variables:
            continue
        break
    replacement = candidates[0][j]['token_str'][1:]
    tree = replaceName(variables[i], replacement).visit(tree)
    variables[i] = replacement

new_code = ast.unparse(tree)
print(CODE)
print(new_code)



def Function(Variable):
    a = sin(Variable)
    b = cos(Variable)
    return a + b

def Function(Variable):
    s = sin(Variable)
    c = cos(Variable)
    return s + c
